In [ ]:
# import packages
import os
import math
import folium
import rasterio
import rioxarray as rxr
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.geometry as sg
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm

from matplotlib import pyplot
from math import radians, cos, sin, asin, sqrt
from folium.plugins import MeasureControl
from IPython.display import display
from shapely.geometry import Polygon
from shapely.geometry import box
from branca.colormap import LinearColormap
from pyproj import CRS, Transformer

In [ ]:
home_directory = os.path.expanduser( '~' )
os.chdir(home_directory + '/DS_Project/modules')

import pickle
import yaml
config_path = 'config.yml'
with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
path = config['data']['data'] + '/uhi_model/'
path_raw = path + 'raw/'
path_visual = path + 'visual/'
path_grid = path + 'grid/'
path_model = path + 'model/'

In [ ]:
from data_retrieval.Ecostress.utils import *
from models.UHI_modeling.disaggregate import *

In [ ]:
grid_size_meters = 250
with open(path_grid + 'grid_' + str(grid_size_meters) + '_e.pkl', 'rb') as file:
    grid = pickle.load(file)
grid.head()

## Exemplary target extraction

In [ ]:
path_tif = config['data']['data'] + '/ECOSTRESS/avgAfterNoon_HW.tif'
src = rasterio.open(path_tif)
lst_array = rxr.open_rasterio(path_tif)

In [ ]:
example_index = 999
example_id = grid.iloc[example_index, grid.shape[1]-1]
subset = grid.iloc[example_index,:].geometry

In [ ]:
clip_all = lst_array.rio.clip([subset], crs=4326, all_touched=True)
clip_inner = lst_array.rio.clip([subset],all_touched=False)

In [ ]:
lst_pixel_map = pixels_to_foliumMap(clip_all, subset, crs='EPSG4326')
lst_pixel_map

In [ ]:
lst_pixel_map = pixels_to_foliumMap(clip_all, subset, crs='EPSG3857')
lst_pixel_map.save(path_visual + 'lst_pixel_example.html')
lst_pixel_map

In [ ]:
temperature_data = clip_all[0].values

fig, ax = plt.subplots()
im = ax.imshow(temperature_data, cmap='spring', aspect='equal')

cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Temperature')

ax.set_title('Temperature Map')
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')

plt.show()

In [ ]:
print(clip_all[0].shape)
print(clip_inner[0].shape)

In [ ]:
naive_pixel_mean(lst_array, subset)

In [ ]:
weights, m = weighted_pixel_mean(lst_array, subset, a=70)
weights

In [ ]:
m

In [ ]:
grid['nLST'] = grid.apply(lambda row: naive_pixel_mean_wrapper(row, lst_array), axis=1)
grid['wLST'] = grid.apply(lambda row: weighted_pixel_mean_wrapper(row, lst_array), axis=1)
grid.head()

## Visualise final results

In [ ]:
with open(path + 'final_' + str(grid_size_meters) + '_e.pkl', 'rb') as file:
    final = pickle.load(file)
final.head()

In [ ]:
lst_map = tiffs_to_foliumMap(path_tif, pixelated=False, minTemp=20, maxTemp=45)

cols = ['id','impervious','building','low vegetation','water','trees','road','avg_height','nLST','wLST']
for idx, grids in final.iterrows():
        values = [f"{column}: {np.round(grids[column], 2)}" for column in cols]
        tooltip_text = "<br>".join(values)
        style = {'fillColor': 'transparent', 'color': 'black', 'colorOpacity': 0.7}

        folium.GeoJson(
                grids.geometry,
                style_function=lambda x: style,
                popup=folium.Popup(tooltip_text, max_width=300),
                name = grids['id'],
                control = False).add_to(lst_map)

lst_map

In [ ]:
lst_map.save(path_visual + 'lst_grid_' + str(grid_size_meters) + '_e.html')